In [1]:
import os
import plaidml.keras
plaidml.keras.install_backend()
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"


In [2]:
from keras.layers import Dense, Dropout, Conv2D, Input, MaxPool2D, Flatten
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers import ELU


In [3]:

def proposed_model(input_h=128, input_w=128):
    input_shape = (input_h, input_w, 3)

    nb_classes = 8
    inputs = Input(input_shape)

    # layer 1
    x = Conv2D(64, (3, 3), strides=(1, 1), kernel_initializer='glorot_uniform')(inputs)
    x = ELU()(x)
    x = BatchNormalization()(x)

    # layer 2
    x = Conv2D(64, (3, 3), strides=(1, 1), kernel_initializer='glorot_uniform')(x)
    x = ELU()(x)
    x = BatchNormalization()(x)

    # layer3
    x = MaxPool2D(pool_size=(2, 2), strides=(2, 2))(x)

    # layer4
    x = Conv2D(128, (3, 3), strides=(1, 1), kernel_initializer='glorot_uniform')(x)
    x = ELU()(x)
    x = BatchNormalization()(x)

    # layer5
    x = Conv2D(128, (3, 3), strides=(1, 1), kernel_initializer='glorot_uniform')(x)
    x = ELU()(x)
    x = BatchNormalization()(x)

    # layer6
    x = MaxPool2D(pool_size=(2, 2), strides=(2, 2))(x)

    # layer7
    x = Conv2D(256, (3, 3), strides=(1, 1), kernel_initializer='glorot_uniform')(x)
    x = ELU()(x)
    x = BatchNormalization()(x)

    # layer 8
    x = Conv2D(256, (3, 3), strides=(1, 1), kernel_initializer='glorot_uniform')(x)
    x = ELU()(x)
    x = BatchNormalization()(x)

    # layer 9
    x = MaxPool2D(pool_size=(2, 2), strides=(2, 2))(x)

    x = Flatten()(x)

    # layer 10
    x = Dense(2048)(x)
    x = ELU()(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(nb_classes, activation='softmax')(x)

    model = Model(inputs, x)
    return model

In [4]:
from keras.callbacks import Callback
import keras.backend as K
import math

class Step(Callback):

    def __init__(self, verbose=0):
        self.verbose = verbose
        self.current_epoch = 1

    def change_lr(self, new_lr):
        K.set_value(self.model.optimizer.lr, new_lr)
        if self.verbose == 1:
            print('Learning rate is %g' %new_lr)

    def on_epoch_begin(self, epoch, logs={}):
        self.current_epoch = epoch
    def on_batch_begin(self, batch, logs={}):
        new_lr = 0.0001 * 0.95 ** math.ceil(self.current_epoch*batch/1000)
        self.change_lr(new_lr)



In [5]:
from keras.optimizers import Adam
from keras.utils import np_utils

import numpy as np
import random
import cv2
import os
import matplotlib
matplotlib.use('AGG')
import matplotlib.pyplot as plt
import glob
import math

import os

In [6]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session



In [7]:

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.45
set_session(tf.Session(config=config))


In [8]:

def plot_history(history, result_dir):
    plt.plot(history.history['acc'], marker='.')
    plt.plot(history.history['val_acc'], marker='.')
    plt.title('model accuracy')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.grid()
    plt.legend(['acc', 'val_acc'], loc='lower right')
    plt.savefig(os.path.join(result_dir, 'model_accuracy.png'))
    plt.close()

    plt.plot(history.history['loss'], marker='.')
    plt.plot(history.history['val_loss'], marker='.')
    plt.title('model loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.grid()
    plt.legend(['loss', 'val_loss'], loc='upper right')
    plt.savefig(os.path.join(result_dir, 'model_loss.png'))
    plt.close()

In [9]:
def save_history(history, result_dir):
    loss = history.history['loss']
    acc = history.history['acc']
    val_loss = history.history['val_loss']
    val_acc = history.history['val_acc']
    nb_epoch = len(acc)

    with open(os.path.join(result_dir, 'result.txt'), 'w') as fp:
        fp.write('epoch\tloss\tacc\tval_loss\tval_acc\n')
        for i in range(nb_epoch):
            fp.write('{}\t{}\t{}\t{}\t{}\n'.format(
                i, loss[i], acc[i], val_loss[i], val_acc[i]))
        fp.close()


In [10]:

def process_batch(lines,img_path,inputH,inputW,train=True):
    imagew = 128
    imageh = 128
    num = len(lines)
    batch = np.zeros((num, imagew, imageh, 3), dtype='float32')


    labels = np.zeros(num, dtype='int')
    for i in range(num):
        path = lines[i].split(' ')[0]
        label = lines[i].split(' ')[-1]

        label = label.strip('\n')
        label = int(label)

        img = os.path.join(img_path, path)

        if train:
            crop_x = random.randint(0, np.max([0, imagew-inputW]))
            crop_y = random.randint(0, np.max([0, imageh-inputH]))
            is_flip = random.randint(0, 1)


            image = cv2.imread(img)

            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = image[crop_y:crop_y + inputH, crop_x:crop_x + inputW, :]
            image = cv2.resize(image, (imagew, imageh))

            if is_flip == 1:
                image = cv2.flip(image, 1)

            #batch[i][:][:][:] = image[crop_y:crop_y + inputH,crop_x:crop_x + inputW, :]
            batch[i] = image
            labels[i] = label
        else:
            image = cv2.imread(img)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)


            Hshmean = int(np.round(np.max([0, np.round((imageh-inputH)/2)])))
            Wshmean = int(np.round(np.max([0, np.round((imagew-inputW)/2)])))
            image = image[Hshmean:Hshmean+inputH,Wshmean:Wshmean+inputW, :]
            image = cv2.resize(image, (imagew, imageh))

            batch[i] = image
            labels[i] = label

    return batch, labels


In [11]:

def generator_train_batch( train_txt, batch_size, num_classes, img_path, inputH, inputW ):
    ff = open(train_txt, 'r')
    lines = ff.readlines()
    num = len(lines)
    while True:
        new_line = []
        index = [n for n in range(num)]
        random.shuffle(index)
        for m in range(num):
            new_line.append(lines[index[m]])

        for i in range(int(num/batch_size)):
            a = i*batch_size
            b = (i+1)*batch_size
            x_train, x_labels = process_batch(new_line[a:b], img_path, inputH, inputW, train=True)
            y = np_utils.to_categorical(np.array(x_labels), num_classes)
            yield x_train, y


In [12]:

def generator_val_batch(val_txt,batch_size,num_classes,img_path,inputH,inputW):
    f = open(val_txt, 'r')
    lines = f.readlines()
    num = len(lines)
    while True:
        new_line = []
        index = [n for n in range(num)]
        random.shuffle(index)
        for m in range(num):
            new_line.append(lines[index[m]])
        for i in range(int(num / batch_size)):
            a = i * batch_size
            b = (i + 1) * batch_size
            y_test,y_labels = process_batch(new_line[a:b],img_path,inputH,inputW,train=False)
            y = np_utils.to_categorical(np.array(y_labels), num_classes)
            yield y_test, y


In [13]:

def generator_train_batch_proposed( new_lines, k, batch_size, num_classes, img_path, inputH, inputW ):

    val_set = 0
    while True:
        if val_set >= k:
            val_set = 0
        else:
            pass

        new_line = []
        for i in range(len(new_lines)):
            if val_set != i:
                new_line += new_lines[i]

        num = len(new_line)
        random.shuffle(new_line)
        for i in range(int(num/batch_size)):
            a = i*batch_size
            b = (i+1)*batch_size
            x_train, x_labels = process_batch(new_line[a:b], img_path, inputH, inputW, train=True)
            y = np_utils.to_categorical(np.array(x_labels), num_classes)
            yield x_train, y
        val_set += 1


In [14]:

def generator_val_batch_proposed(new_lines, k, batch_size, num_classes, img_path, inputH, inputW):


    val_set = 0
    while True:
        if val_set >= k:
            val_set = 0
        else:
            pass

        new_line = new_lines[val_set]
        num = len(new_lines)
        random.shuffle(new_line)

        for i in range(int(num / batch_size)):
            a = i * batch_size
            b = (i + 1) * batch_size
            y_test,y_labels = process_batch(new_line[a:b],img_path,inputH,inputW,train=False)
            y = np_utils.to_categorical(np.array(y_labels), num_classes)
            yield y_test, y
        val_set += 1

In [15]:
outputdir = 'result/first_attempt/'
if os.path.isdir(outputdir):
    print('save in :'+outputdir)
else:
    os.makedirs(outputdir)


save in :result/first_attempt/


In [16]:
train_img_path = 'MIT-BIH_AD/'
train_file = 'MIT-BIH_AD_train.txt'
num_classes = 8
k = 10

In [17]:
f1 = open(train_file, 'r')
lines = f1.readlines()
f1.close()


In [18]:
train_samples = len(lines)
val_samples = len(lines)//k


In [19]:
num = len(lines)
new_lines = []
index = [n for n in range(num)]
random.shuffle(index)
for m in range(num):
    new_lines.append(lines[index[m]])


In [20]:

lines = new_lines
temp = []
new_lines = []
for i in range(num):
    if i % val_samples == 0:
        temp = []
        new_lines.append(temp)
    temp.append(lines[i])

In [21]:
batch_size = 1
epochs = 40
input_h = 96
input_w = 96

In [22]:
model = proposed_model()

INFO:plaidml:Opening device "metal_amd_radeon_pro_5300m.0"


In [23]:
lr = 0.0001
adam = Adam(lr=lr)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [24]:
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 126, 126, 64)      1792      
_________________________________________________________________
elu_1 (ELU)                  (None, 126, 126, 64)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 126, 126, 64)      256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 124, 124, 64)      36928     
_________________________________________________________________
elu_2 (ELU)                  (None, 124, 124, 64)      0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 124, 124, 64)      256       
__________

In [25]:
history = model.fit_generator(generator_train_batch_proposed(new_lines, k, batch_size, num_classes, train_img_path, input_h, input_w),
                              steps_per_epoch=train_samples // batch_size,
                              epochs=epochs,
                              callbacks=[Step()],
                              validation_data=generator_val_batch_proposed(new_lines, k, batch_size, num_classes, train_img_path, input_h, input_w),
                              validation_steps=val_samples // batch_size,
                              verbose=1)

Epoch 1/40
  695/62052 [..............................] - ETA: 3:30:25 - loss: 1.3499 - acc: 0.7122

KeyboardInterrupt: 